In [7]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [8]:
!pip install ollama

In [9]:
#starting ollama server locally
import subprocess
import time
process = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)  # Wait for 5 seconds

In [10]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕▏  12 KB                         
pulling 8ab4849b038c... 100% ▕▏  254 B                         
pulling 577073ffcc6c... 100% ▕▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [11]:
!ollama list

NAME             ID              SIZE      MODIFIED       
llama3:latest    365c0bd3c000    4.7 GB    14 seconds ago    


In [12]:
import ollama

# Correct message format
response = ollama.chat(model="llama3", messages=[{"role": "user", "content": "What is machine learning?"}])

# Print the response from the model
print(response.message.content)


Machine learning (ML) is a subfield of artificial intelligence (AI) that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. In other words, ML allows computers to improve their performance on a task over time, based on the data they receive.

Here's how it works:

1. **Training**: You feed a machine learning algorithm with a dataset containing examples of input (e.g., images, text, audio) and corresponding output (e.g., labels, classifications).
2. **Model building**: The algorithm uses this training data to build a mathematical model that can make predictions or decisions.
3. **Testing**: You test the model on a separate dataset to evaluate its performance.
4. **Improvement**: If the model performs well, you refine it by adjusting parameters, adding more data, or using different algorithms. This process is called "iteration."
5. **Deployment**: Once the model is trained and tested satisfactorily, you deploy it in a re

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd

file_path = '/content/drive/My Drive/ARM/financial_statement.xlsx'
df = pd.read_excel(file_path)  # Read the first sheet
print(df)  # Display the first few rows

                  Particulars  2024 (in $ millions)  2023 (in $ millions)
0                     Revenue               50000.0               47000.0
1   Cost of Goods Sold (COGS)              -30000.0              -28500.0
2                Gross Profit               20000.0               18500.0
3          Operating Expenses                   NaN                   NaN
4         Selling & Marketing               -3500.0               -3200.0
5    General & Administrative               -4000.0               -3800.0
6                R&D Expenses               -2500.0               -2300.0
7            Operating Income               10000.0                9200.0
8    Other Income / (Expense)                   NaN                   NaN
9            Interest Expense                -500.0                -600.0
10               Other Income                 300.0                 200.0
11      Net Income Before Tax                9800.0                8800.0
12         Income Tax Expense         

In [15]:
print(df.iloc[0])
# read revenue data

Particulars             Revenue
2024 (in $ millions)    50000.0
2023 (in $ millions)    47000.0
Name: 0, dtype: object


In [16]:
!pip install pymupdf

In [17]:
import fitz  # PyMuPDF

pdf_document = "/content/drive/My Drive/ARM/10k.pdf"
tenk_text = ""

with fitz.open(pdf_document) as doc:
    for page in doc:
        tenk_text += page.get_text()

print(tenk_text)

Excerpt from 10-K Filing (MD&A Section) 
XYZ Corporation – Management’s Discussion and Analysis (MD&A) 
Revenue Growth: 
Revenue increased by 6.38% ($3B increase) due to higher sales volume in 
international markets and improved pricing strategies. 
Gross Profit: 
Gross profit improved due to a reduction in supply chain costs and improved 
operational efficiencies. However, raw material costs were higher in Q2, which 
slightly offset the gain. 
Operating Expenses: 
 
Selling & Marketing Expenses: Increased due to higher advertising spend 
and expansion into new markets. 
 
General & Administrative Expenses: Increased primarily due to higher 
employee salaries and office lease costs. 
 
R&D Expenses: Increased as the company continued investing in new 
product innovations, especially in AI-powered solutions. 
Net Income: 
Net income increased by 11.36% ($800M) due to higher revenue and operational 
efficiencies, offset by higher interest expenses and taxes. 
Key Reasons for Profit In

In [18]:
# Combine structured financial data and unstructured 10-K insights
combined_text = (
    f"Particular: {df.iloc[0]['Particulars']}, "
    f"Revenue 2024: {df.iloc[0]['2024 (in $ millions)']}, "
    f"Revenue 2023: {df.iloc[0]['2023 (in $ millions)']}. "
    f"Additional Info from 10-K: {tenk_text}"
)

print(combined_text)  # Preview the final combined text

Particular: Revenue, Revenue 2024: 50000.0, Revenue 2023: 47000.0. Additional Info from 10-K: Excerpt from 10-K Filing (MD&A Section) 
XYZ Corporation – Management’s Discussion and Analysis (MD&A) 
Revenue Growth: 
Revenue increased by 6.38% ($3B increase) due to higher sales volume in 
international markets and improved pricing strategies. 
Gross Profit: 
Gross profit improved due to a reduction in supply chain costs and improved 
operational efficiencies. However, raw material costs were higher in Q2, which 
slightly offset the gain. 
Operating Expenses: 
 
Selling & Marketing Expenses: Increased due to higher advertising spend 
and expansion into new markets. 
 
General & Administrative Expenses: Increased primarily due to higher 
employee salaries and office lease costs. 
 
R&D Expenses: Increased as the company continued investing in new 
product innovations, especially in AI-powered solutions. 
Net Income: 
Net income increased by 11.36% ($800M) due to higher revenue and opera

In [19]:
!pip install chromadb

In [20]:
import ollama
import chromadb

# Generate embeddings using Llama3
response = ollama.embeddings(model="llama3", prompt=combined_text)
embedding_vector = response["embedding"]

# Store in ChromaDB
chroma_client = chromadb.PersistentClient(path="./vector_db")
collection = chroma_client.get_or_create_collection(name="financial_reports")

collection.add(
    ids=["revenue_10k_data"],
    embeddings=[embedding_vector],
    metadatas=[{"source": "financial_data_10K"}],
    documents=[combined_text]
)

print("Revenue and 10-K data successfully stored in vector database!")


Revenue and 10-K data successfully stored in vector database!


In [23]:
# # Ensure retrieved_doc is a string (if it's a list, join its contents)
# retrieved_doc = " ".join(retrieved_doc) if isinstance(retrieved_doc, list) else retrieved_doc

# # Ask Llama3 to generate a response
# response = ollama.chat(model="llama3", messages=[
#     {"role": "system", "content": "You are a financial assistant providing insights on revenue changes."},
#     {"role": "user", "content": retrieved_doc}  # Make sure this is a string
# ])

# print(response["message"]["content"])

# Define the user query (simplified to focus on revenue insights)
user_query = "Provide insights on the revenue."

# Generate query embedding
query_embedding = ollama.embeddings(model="llama3", prompt=user_query)["embedding"]

# Retrieve the most relevant document from ChromaDB
results = collection.query(query_embeddings=[query_embedding], n_results=1)

# Ensure the retrieved document is available
if "documents" in results and results["documents"]:
    retrieved_doc = results["documents"][0]  # Extract the first document
else:
    raise ValueError("No relevant document retrieved from ChromaDB.")

# Ensure retrieved_doc is a string (if it's a list, join its contents)
retrieved_doc = " ".join(retrieved_doc) if isinstance(retrieved_doc, list) else str(retrieved_doc)

# Ask Llama3 to generate a response
response = ollama.chat(model="llama3", messages=[
    {"role": "system", "content": "You are a financial assistant providing insights on revenue."},
    {"role": "user", "content": retrieved_doc}  # Make sure this is a string
])

# Print the response
print(response["message"]["content"])




Based on the information provided, I can provide some insights on the revenue changes for XYZ Corporation in 2024 compared to 2023.

Firstly, the company's revenue increased by 6.38% from $47,000 in 2023 to $50,000 in 2024, which is a significant growth of $3,000. This increase can be attributed to higher sales volume in international markets and improved pricing strategies.

In terms of the factors contributing to this revenue growth, I would highlight the following:

1. International expansion: The company's sales grew in Asia and Europe, indicating a successful expansion into new markets.
2. Pricing strategies: Improved pricing strategies likely helped increase revenue by optimizing prices and driving sales.

These factors have contributed to a 6.38% increase in revenue, which is a positive trend for the company.

It will be important for the company to continue to focus on these growth drivers and potentially explore other opportunities to drive revenue growth in the future.

Addit